In [1]:
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 kB 832.8 kB/s eta 0:00:00a 0:00:01


In [2]:
!pip install opentelemetry-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 957.6 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 2.0 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install opentelemetry-exporter-otlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 499.6 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 7.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 5.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.3
    Uninstalling protobuf-4.24.3:
      Successfully uninstalled protobuf-4.24.3


In [4]:
import json
import random
import time
import uuid
from datetime import datetime, timezone

from kafka import KafkaProducer

# ---------------- OpenTelemetry ----------------
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import (
    OTLPSpanExporter,
)
from opentelemetry.trace import SpanKind, TraceFlags
from opentelemetry.trace import set_span_in_context
from opentelemetry.trace import SpanContext, NonRecordingSpan

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------

KAFKA_BOOTSTRAP_SERVERS = "kafka:9092"
KAFKA_TOPIC = "raw-events"

# OpenTelemetry Collector endpoint
OTLP_ENDPOINT = "jaeger:4317"  # gRPC
SERVICE_NAME = "windows-log-producer"

EVENT_INTERVAL_SEC = (1, 5)

# ---------------------------------------------------------------------
# Tracing setup (OTLP)
# ---------------------------------------------------------------------

resource = Resource.create(
    {
        "service.name": SERVICE_NAME,
    }
)

trace.set_tracer_provider(TracerProvider(resource=resource))
tracer = trace.get_tracer(__name__)

otlp_exporter = OTLPSpanExporter(
    endpoint=OTLP_ENDPOINT,
    insecure=True,  # внутри docker-сети
)

span_processor = BatchSpanProcessor(otlp_exporter)
trace.get_tracer_provider().add_span_processor(span_processor)

# ---------------------------------------------------------------------
# Kafka producer
# ---------------------------------------------------------------------

producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode("utf-8"),
)

# ---------------------------------------------------------------------
# Event generation
# ---------------------------------------------------------------------

USERS = ["alice", "bob", "charlie", "admin"]
HOSTS = ["win-01", "win-02", "win-03"]
IPS = ["10.0.0.10", "10.0.0.11", "10.0.0.12"]

PROCESS_SCENARIOS = [
    {
        "process_name": "powershell.exe",
        "command_line": "powershell -EncodedCommand SQBFAFgA",
        "parent_process": "explorer.exe",
    },
    {
        "process_name": "cmd.exe",
        "command_line": "cmd.exe /c whoami",
        "parent_process": "explorer.exe",
    },
    {
        "process_name": "notepad.exe",
        "command_line": "notepad.exe",
        "parent_process": "explorer.exe",
    },
]

LOGIN_SCENARIOS = [
    {"logon_type": "success"},
    {"logon_type": "failure"},
]

def generate_event():
    event_id = str(uuid.uuid4())
    event_type = random.choice(["process_start", "user_login"])

    event = {
        "event_id": event_id,
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "user": random.choice(USERS),
        "host": random.choice(HOSTS),
        "source_ip": random.choice(IPS),
        "event_type": event_type,
    }

    if event_type == "process_start":
        s = random.choice(PROCESS_SCENARIOS)
        event.update(
            {
                "process_name": s["process_name"],
                "command_line": s["command_line"],
                "parent_process": s["parent_process"],
                "logon_type": None,
            }
        )
    else:
        s = random.choice(LOGIN_SCENARIOS)
        event.update(
            {
                "process_name": None,
                "command_line": None,
                "parent_process": None,
                "logon_type": s["logon_type"],
            }
        )

    return event

# ---------------------------------------------------------------------
# Main loop
# ---------------------------------------------------------------------

print("Starting Windows log producer (OTLP)...")
print(f"Kafka topic: {KAFKA_TOPIC}")
print(f"OTLP endpoint: {OTLP_ENDPOINT}")

while True:
    event = generate_event()
    event_id = event["event_id"]

    # event_id → trace_id (UUID → 128-bit int)
    trace_id = uuid.UUID(event_id).int

    parent_ctx = set_span_in_context(
        NonRecordingSpan(
            SpanContext(
                trace_id=trace_id,
                span_id=random.getrandbits(64),
                is_remote=False,
                trace_flags=TraceFlags(TraceFlags.SAMPLED),
                trace_state={},
            )
        )
    )

    with tracer.start_as_current_span(
        "produce_event",
        context=parent_ctx,
        kind=SpanKind.PRODUCER,
    ) as span:

        span.set_attribute("event.id", event_id)
        span.set_attribute("event.type", event["event_type"])
        span.set_attribute("host.name", event["host"])
        span.set_attribute("user.name", event["user"])

        with tracer.start_as_current_span("kafka_produce"):
            producer.send(KAFKA_TOPIC, event)
            producer.flush()

        print(f"Produced event {event_id} ({event['event_type']})")

    time.sleep(random.uniform(*EVENT_INTERVAL_SEC))


Starting Windows log producer (OTLP)...
Kafka topic: raw-events
OTLP endpoint: jaeger:4317
Produced event 9febcea8-030a-4531-a743-65ac932e4307 (process_start)
Produced event babd5bb8-cf60-42a8-b0ab-a11ba7e46e6f (process_start)
Produced event 4aa5a372-0f2f-4c76-99a7-5d473f5baafa (user_login)
Produced event 52442bed-403c-4cbc-95e6-0752d189f3de (process_start)
Produced event 67c7fe57-42cd-41bb-94b8-c6aac3466412 (process_start)
Produced event 5c74d962-9ae5-43a7-82a3-eaf4230fc929 (user_login)
Produced event 0e92d8aa-881c-4df4-ad19-5da29ca97dbe (process_start)
Produced event d8ba120a-4531-41ec-8e0b-a759fa0eef77 (process_start)
Produced event a9a37b05-e2eb-498c-852f-eff05cd0065c (user_login)
Produced event c50a30b9-b337-4d07-a787-cb519735aa54 (process_start)
Produced event d1dd9cca-b548-4ec4-a092-02b4f3493e49 (process_start)
Produced event be30f83e-dce2-44af-86f3-9f75224ed8c5 (process_start)
Produced event 9bfbe731-eb6a-42a9-be90-a8f21cd9a60c (user_login)
Produced event f6673605-1651-4e75-bb2

KeyboardInterrupt: 